In [20]:
import threading
import multiprocessing
from threading import Thread, current_thread, Event
from multiprocessing import Process, current_process, Pool
from multiprocessing.connection import Listener, Client
from IPython import display
import pandas as pd
from datetime import datetime as dt
from pandas.tseries.offsets import BDay, Day
from dataclasses import dataclass, fields, asdict, replace
import itertools
import datetime
import signal
# from matplotlib import pyplot as plt

In [ ]:
import yfinance as yf
import pandas_ta as ta
import time, os, sys
import numpy as np
import import_ipynb
import pickle
# from tqdm.notebook import tqdm

In [ ]:
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed

In [ ]:
from india_calendar import IBDay

In [ ]:
class BackFeed():
    def __init__(self,tickers=['WIPRO.NS','TCS.NS'],nw=1,nd=0,delay=0):
        todayS=dt.today().strftime("%d-%b-%Y")
        todayD=pd.to_datetime(todayS)
        todayD=todayD-BDay(1)+BDay(1)
        todayS=todayD.strftime("%d-%b-%Y")
        self.tickers=tickers
        self.data={}
        self.counter=0
        self.delay=delay
        self.dtD={}
        self.ncounter={t:0 for t in self.tickers}
        self.offsets={t:{} for t in self.tickers}
        for t in self.tickers:
            dfL=[]
            try:
                for d in range(nw,0,-1):
                    if nd==0:
                        df=yf.Ticker(t).history(period='1d',interval='1m')
                        df['ticker']=t
                        df['datetime']=df.index
                        #df=add_indicators(df)
                        dfL=[df]
                        break
                    # if nw==4: start=(todayD-BDay(nd*d-1)).strftime("%Y-%m-%d")
                    # else: start=(todayD-BDay(nd*d)).strftime("%Y-%m-%d")
                    start=(todayD-BDay(nd*d)).strftime("%Y-%m-%d")
                    end=(todayD-BDay(nd*(d-1))).strftime("%Y-%m-%d")
                    # print(start,end)
                    df=yf.Ticker(t).history(start=start, end=end,interval='1m')
                    #df=add_indicators(df)
                    if df.shape[0]>450: 
                        # self.data[t]=df
                        df=df.fillna(0)
                        # self.dtD[t]=df.iloc[-1]['Date']
                        df['Date']=df.index.strftime('%d-%b-%Y')
                        df['ticker']=t
                        df['datetime']=df.index
                        dfL=dfL+[df]
                self.data[t]=pd.concat(dfL,axis=0)
                self.dtD[t]=self.data[t].iloc[0]['Date']
            except:
                pass
        self.tickers=[t for t in self.data]
        self.maxcount=max([self.data[t].shape[0] for t in self.tickers])
        self.df=pd.concat([self.data[t] for t in self.tickers])
        if self.df.loc[self.df['Date']==todayS].shape[0]>0:
            self.df.drop(self.df.loc[self.df['Date']==todayS].index,inplace=True)
        self.dates=self.df.Date.unique()[1:]
    def init_counters(self,date1=None):
        t0=self.tickers[0]
        if date1==None: date1=self.data[t0]['Date'].unique()[1]
        self.counter=self.offsets[t0][date1]
        for t in self.tickers:
            date1=self.data[t]['Date'].unique()[1]
            self.ncounter[t]=self.offsets[t][date1]
        self.dtD={t:self.get(t)['Date'] for t in self.tickers}
    def step(self):
        dtD_old=self.dtD
        self.counter+=1
        self.dtD={t:self.get(t)['Date'] for t in self.tickers}
        for t in self.tickers:
            if self.dtD[t]!=dtD_old[t]: 
                self.ncounter[t]=self.offsets[t][self.dtD[t]]
            else: self.ncounter[t]+=1
        time.sleep(self.delay)
    def get(self,t):
        if self.counter<self.data[t].shape[0]: idx=self.counter
        else: idx=-1
        return self.data[t].iloc[idx]
    def getData(self,t):
        if self.counter<self.data[t].shape[0]:
            return self.data[t].iloc[0:self.counter+1]
        else: return self.data[t]
    def getBasket(self,client=None):
        getfL=[self.get(t) for t in self.tickers]
        dfL=[pd.DataFrame([getf.values],columns=getf.index) for getf in getfL]
        return pd.concat(dfL,axis=0)
    def check_done(self):
        if self.counter==self.maxcount: return True
        else: return False
    def getN(self,t,d):
        if self.ncounter[t]<self.ndata[t][d].shape[0]: idx=self.ncounter[t]
        else: idx=-1
        return self.ndata[t][d].iloc[idx]
    def getDataN(self,t,d):
        if self.ncounter[t]<self.ndata[t][d].shape[0]:
            return self.ndata[t][d].iloc[0:self.ncounter[t]+1]
        else: return self.ndata[t][d]

In [ ]:
class LiveFeed():
    def __init__(self,tickers=['WIPRO.NS','TCS.NS'],period='2d',interval='1m',delay=1):
        self.tickers=tickers
        self.data={}
        self.period=period
        self.interval=interval
        self.delay=delay
        self.counter=0
        self.dtD={}
        self.offsets={t:{} for t in self.tickers}
        todayS=dt.today().strftime("%d-%b-%Y")
        self.todayS=todayS
        todayD=pd.to_datetime(todayS)
        self.todayD=todayD-IBDay(1)+IBDay(1)
        self.step(init=True)
    def init_counters(self):
        t0=self.tickers[0]
        date1=self.data[t0]['Date'].unique()[1]
        self.counter=self.offsets[t0][date1]
        self.dtD={t:self.get(t)['Date'] for t in self.tickers}
    def step(self,init=False):
        self.counter+=1
        for t in self.tickers:
            try:
                # df=yf.Ticker(t).history(self.period,self.interval)
                end=(self.todayD+IBDay(1)).strftime("%Y-%m-%d")
                start=(self.todayD-IBDay(1)).strftime("%Y-%m-%d")
                df=yf.Ticker(t).history(start=start, end=end,interval='1m')
                df['Date']=df.index.strftime('%d-%b-%Y')
                df['ticker']=t
                df['datetime']=df.index
                if df.shape[0]>100:
                    df=df.fillna(0)
                    self.data[t]=df
                    self.dtD[t]=df.iloc[-1]['Date']
            except:
                pass
        if init==True: 
            self.tickers=[t for t in self.data]
            self.df=pd.concat([self.data[t] for t in self.tickers])
            self.dates=self.df.Date.unique()[1:]
        add_ta_features_feed(self)
        add_sym_feature_feed(self,tickers=self.tickers,live=True)
        if init==False: time.sleep(self.delay)
    def getData(self,t):
        return self.data[t]
    def get(self,t):
        return self.data[t].iloc[-1]
    def getBasket(self):
        getfL=[self.get(t) for t in self.tickers]
        dfL=[pd.DataFrame([getf.values],columns=getf.index) for getf in getfL]
        return pd.concat(dfL,axis=0)
    def getDataN(self,t,d):
        return self.ndata[t][d]
    def getN(self,t,d):
        return self.ndata[t][d].iloc[-1]
    def check_done(self):
        return False

In [ ]:
class DataFeed():
    def __init__(self,tickers=['WIPRO.NS','TCS.NS'],
                 datafile='./DataLocal/algo_fin_new/labeled_data_02-Mar-2022.csv',
                dfgiven=False,df=None,delay=0):
        self.tickers=tickers
        self.data={}
        self.counter=0
        self.delay=delay
        self.dtD={}
        self.ncounter={t:0 for t in self.tickers}
        self.offsets={t:{} for t in self.tickers}
        if dfgiven==False: self.df=pd.read_csv(datafile)
        else: self.df=df
        self.df['datetime']=pd.to_datetime(self.df['Datetime'])
        self.dates=df.Date.unique()[1:]
        for t in self.tickers:
            self.data[t]=self.df.loc[self.df['ticker']==t]
            self.data[t].index=pd.to_datetime(self.data[t].datetime)
        self.maxcount=min([self.data[t].shape[0] for t in self.tickers])
    def init_counters(self,date1=None):
        t0=self.tickers[0]
        idx=1
        if date1==None:
            counter,idx=0,0
            while counter==0:
                idx+=1
                date1=self.data[t0]['Date'].unique()[idx]
                counter=self.offsets[t0][date1]
            self.counter=counter
        else: self.counter=self.offsets[t0][date1]
        for t in self.tickers:
            date1=self.data[t]['Date'].unique()[1]
            self.ncounter[t]=self.offsets[t][date1]
        self.dtD={t:self.get(t)['Date'] for t in self.tickers}
    def step(self):
        dtD_old=self.dtD
        self.counter+=1
        self.dtD={t:self.get(t)['Date'] for t in self.tickers}
        for t in self.tickers:
            if self.dtD[t]!=dtD_old[t]: 
                self.ncounter[t]=self.offsets[t][self.dtD[t]]
            else: self.ncounter[t]+=1
        time.sleep(self.delay)
    def get(self,t):
        if self.counter<self.data[t].shape[0]: idx=self.counter
        else: idx=-1
        return self.data[t].iloc[idx]
    def getData(self,t):
        if self.counter<self.data[t].shape[0]:
            return self.data[t].iloc[0:self.counter+1]
        else: return self.data[t]
    def getBasket(self,client=None):
        getfL=[self.get(t) for t in self.tickers]
        dfL=[pd.DataFrame([getf.values],columns=getf.index) for getf in getfL]
        return pd.concat(dfL,axis=0)
    def check_done(self):
        if self.counter==self.maxcount: return True
        else: return False
    def getN(self,t,d):
        if self.ncounter[t]<self.ndata[t][d].shape[0]: idx=self.ncounter[t]
        else: idx=-1
        return self.ndata[t][d].iloc[idx]
    def getDataN(self,t,d):
        if self.ncounter[t]<self.ndata[t][d].shape[0]:
            return self.ndata[t][d].iloc[0:self.ncounter[t]+1]
        else: return self.ndata[t][d]

In [ ]:
def clean_feed(feed,ticker):
    feed_dates=feed.data[ticker].Date.unique()
    if feed.data[ticker].loc[feed.data[ticker]['Date']==feed_dates[-1]].shape[0]<10:
        feed.data[ticker]=feed.data[ticker].drop(
                feed.data[ticker].index[-1])

In [ ]:
def clean_feed_tickers(feed,limit=100):
    dropticks=[]
    for t in feed.tickers:
        dfa=feed.data[t]
        for d in dfa['Date'].unique():
            pdt=pd.to_datetime(d)
            pdtp=pdt-BDay(1)
            df=dfa.loc[(pd.to_datetime(dfa['Date'])<=pdt)&
                       (pd.to_datetime(dfa['Date'])>=pdtp)]
            if len(df)<limit: dropticks+=[t]
    feed.tickers=[t for t in feed.tickers if t not in dropticks]
    return feed

Debugging